In [1]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image as tf_image
from sklearn.metrics import classification_report, f1_score, accuracy_score
import matplotlib.pyplot as plt

In [2]:
# ==============================================================================
# Step 1: Configuration & Environment Setup
# ==============================================================================

# Define the path to your dataset folder.
# The folder should contain subfolders for each class (e.g., 'Cassava___healthy').
DATA_DIR = './dataset'

# Define image dimensions and batch size.
IMG_HEIGHT = 224
IMG_WIDTH = 224
BATCH_SIZE = 32

# Path for a user-provided image to classify.
# This variable will be updated later with user input.
user_image_path = ''

# Check if the dataset directory exists.
if not os.path.exists(DATA_DIR):
    print(f"Error: Dataset directory '{DATA_DIR}' not found.")
    print("Please make sure you have the 'dataset' folder with the subfolders "
          "('Cassava___bacterial_blight', etc.) in the same directory as this script.")
    exit()

In [3]:
# ==============================================================================
# Step 2: Data Loading and Pre-processing
# ==============================================================================
print("Loading and pre-processing data...")

# Use ImageDataGenerator for data augmentation and pre-processing.
# Rescale pixel values from [0, 255] to [0, 1].
datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2,  # Use 20% of data for validation
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Load training data.
train_ds = datagen.flow_from_directory(
    DATA_DIR,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='training'
)

# Load validation data.
val_ds = datagen.flow_from_directory(
    DATA_DIR,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='validation'
)

# Get the class names from the dataset.
class_names = list(train_ds.class_indices.keys())
print(f"Found {len(class_names)} classes: {class_names}")

Loading and pre-processing data...
Found 17120 images belonging to 5 classes.
Found 4277 images belonging to 5 classes.
Found 5 classes: ['Cassava___bacterial_blight', 'Cassava___brown_streak_disease', 'Cassava___green_mottle', 'Cassava___healthy', 'Cassava___mosaic_disease']


In [4]:
# ==============================================================================
# Step 3: Build the CNN Model
# ==============================================================================
print("Building the CNN model...")

model = models.Sequential([
    # Input layer and first convolutional block
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(IMG_HEIGHT, IMG_WIDTH, 3)),
    layers.MaxPooling2D((2, 2)),
    # Second convolutional block
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    # Third convolutional block
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    # Fourth convolutional block
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),

    # Flatten the 3D output to 1D for the dense layers.
    layers.Flatten(),
    # Fully connected layers
    layers.Dense(512, activation='relu'),
    layers.Dropout(0.5), # Add dropout to prevent overfitting
    layers.Dense(len(class_names), activation='softmax') # Output layer
])

# Compile the model with an optimizer, loss function, and metrics.
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.summary()

Building the CNN model...



Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 222, 222, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2  (None, 111, 111, 32)      0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 109, 109, 64)      18496     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 54, 54, 64)        0         
 g2D)                                                            
                                                                 
 conv2d_2 (Conv2D)           (None, 52, 52, 128)       73856     
                                                                 
 max_pooling2d_2 (MaxPoolin

In [ ]:
# ==============================================================================
# Step 4: Train the Model
# ==============================================================================
print("Training the model...")

history = model.fit(
    train_ds,
    epochs=30,  # You can adjust the number of epochs
    validation_data=val_ds
)

# Plot training history for visualization
def plot_training_history(history):
    acc = history.history['accuracy']
    val_acc = history.history['val_accuracy']
    loss = history.history['loss']
    val_loss = history.history['val_loss']
    epochs = range(len(acc))

    plt.figure(figsize=(12, 5))
    plt.subplot(1, 2, 1)
    plt.plot(epochs, acc, 'bo', label='Training accuracy')
    plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
    plt.title('Training and Validation Accuracy')
    plt.legend()

    plt.subplot(1, 2, 2)
    plt.plot(epochs, loss, 'ro', label='Training loss')
    plt.plot(epochs, val_loss, 'r', label='Validation loss')
    plt.title('Training and Validation Loss')
    plt.legend()
    plt.show()

# plot_training_history(history)

Training the model...
Epoch 1/30


 42/535 [=>............................] - ETA: 21:49 - loss: 1.2867 - accuracy: 0.5938

In [ ]:
# ==============================================================================
# Step 5: Evaluate the Model (Accuracy, F1 Score)
# ==============================================================================
print("\nEvaluating the model on the validation set...")

# Get predictions and true labels from the validation set
val_labels = []
val_predictions = []
for i in range(len(val_ds)):
    batch = val_ds.next()
    labels = np.argmax(batch[1], axis=1)
    preds = np.argmax(model.predict(batch[0]), axis=1)
    val_labels.extend(labels)
    val_predictions.extend(preds)

# Convert to numpy arrays for calculation
val_labels = np.array(val_labels)
val_predictions = np.array(val_predictions)

# Calculate and print metrics.
accuracy = accuracy_score(val_labels, val_predictions)
f1 = f1_score(val_labels, val_predictions, average='weighted')
report = classification_report(val_labels, val_predictions, target_names=class_names)

print(f"\nModel Accuracy: {accuracy:.4f}")
print(f"Model F1 Score: {f1:.4f}")
print("\nClassification Report:\n", report)

In [ ]:
# ==============================================================================
# Step 6: Prediction on a New Image
# ==============================================================================

def predict_new_image(image_path):
    """
    Loads and pre-processes a single image for prediction.

    Args:
        image_path (str): The file path to the image.

    Returns:
        tuple: Predicted class name and confidence score.
    """
    if not os.path.exists(image_path):
        print(f"Error: The image file '{image_path}' does not exist.")
        return None, None

    # Load the image and resize it to the target size.
    img = tf_image.load_img(image_path, target_size=(IMG_HEIGHT, IMG_WIDTH))
    # Convert the image to a numpy array.
    img_array = tf_image.img_to_array(img)
    # Expand dimensions to create a batch of size 1.
    img_array = np.expand_dims(img_array, axis=0)
    # Rescale pixel values from [0, 255] to [0, 1].
    img_array /= 255.0

    # Make the prediction.
    predictions = model.predict(img_array)
    # Get the index of the class with the highest probability.
    predicted_class_index = np.argmax(predictions[0])
    # Get the confidence score (probability).
    confidence = np.max(predictions[0])

    # Get the predicted class name from the class_names list.
    predicted_class_name = class_names[predicted_class_index]

    return predicted_class_name, confidence

# Example Usage:
print("\n-------------------------------------------------------------")
print("Enter the path to a cassava leaf image to get its prediction.")
print("Example: 'data/test_image.jpg'")
#user_image_path = C:\\Users\\SLM_INFOTECH\\Documents\\Anaconda\\Tapioca\\dataset\\Cassava___green_mottle\\22116035.jpg
user_image_path = input("Image Path: ")
predicted_class, confidence_score = predict_new_image(user_image_path)

if predicted_class:
    print("\nPrediction Results:")
    print(f"The model predicts the leaf image has: {predicted_class}")
    print(f"Confidence: {confidence_score:.2f}")
